<a href="https://colab.research.google.com/github/yougoal/leejunghoon/blob/master/Dqn2013r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import gym
import numpy as np
import random as ran

env = gym.make('CartPole-v1')

# 꺼내서 사용할 리플레이 갯수
REPLAY = 2
# 리플레이를 저장할 리스트
REPLAY_MEMORY = []
# 미니배치
MINIBATCH = 3

INPUT = env.observation_space.shape[0]
OUTPUT = env.action_space.n

# 하이퍼파라미터
LEARNING_LATE = 0.01
NUM_EPISODE = 10

DISCOUNT = 0.99


# 네트워크 구성
x=tf.placeholder(dtype=tf.float32, shape=(1,4))
W1 = tf.get_variable("W1", shape=[4, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
#W1 = tf.get_variable('W1',shape=[INPUT,10],initializer=tf.contrib.layers.xavier_initializer())
#W2 = tf.get_variable('W4',shape=[10, OUTPUT],initializer=tf.contrib.layers.xavier_initializer())
#L1=tf.nn.tanh(tf.matmul(x,W1))
#Q_pre = tf.matmul(L1,W2)
Q_pre = tf.matmul(x, W1)
y=tf.placeholder(dtype=tf.float32, shape=(1, env.action_space.n))

# 손실 함수
loss = tf.reduce_sum(tf.square(y-Q_pre))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_LATE)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
rList=[]

with tf.Session() as sess:
    sess.run(init)
    for episode in range(10):
        start = env.reset()
        e = 1. / ((episode/25)+1)
        rall = 0
        d = False
        count=0
        while not d:
            # env.render()
            count+=1
            # 현재 상태(s)로 Q값을 예측
            s_t = np.reshape(start,[1,INPUT])
            Q = sess.run(Q_pre, feed_dict={x:s_t})
            print("Q(st,at): {}".format(Q))
            print("Q(st,a)를 구하는 W값")
            print(sess.run([W1]))
            # e-greedy 를 사용하여 action값 구함
            if e > np.random.rand(1):
                a = env.action_space.sample()
            else:
                a = np.argmax(Q)
            # action을 취함
            s1, r, d, _ = env.step(a)
            print("s_t: {} action: {} Reward{} ".format(s_t,a,r))
            print("st+1: {}  done{}".format(s1,d))
            # state, action, reward, next_state, done 을 메모리에 저장
            REPLAY_MEMORY.append([s_t,a,r,s1,d])
            print("메모리",REPLAY_MEMORY[0:1])
            print("메모리",REPLAY_MEMORY[1:2])
            print("메모리",REPLAY_MEMORY[2:3])
            print("메모리",REPLAY_MEMORY[3:4])
            print("메모리",REPLAY_MEMORY[4:5])
            print("메모리",REPLAY_MEMORY[5:6])
            # 메모리에 50000개 이상의 값이 들어가면 가장 먼저 들어간 것부터 삭제
            if len(REPLAY_MEMORY) > 5:
                del REPLAY_MEMORY[0]
            rall += r
            s = s1
        # 10 번의 스탭마다 미니배치로 학습
            print(" EPISODE",episode)
        if episode % 10== 5 :
            for i in range(MINIBATCH):
                # 메모리에서 사용할 리플레이를 랜덤하게 가져옴
                for sample in ran.sample(REPLAY_MEMORY, REPLAY):
                  s_t_r, a_r, r_r, s1_r ,d_r = sample
                  print("Ms_t: {} Maction: {} MReward{} ".format(s_t_r,a_r,r_r))
                  print(" MSt+1 {} done{}".format(s1_r,d_r))
                  if d_r:
                    Q[0, a_r] = -100
                  else:
                    s1_t_r= np.reshape(s1_r,[1,4])
                    Q_next = sess.run(Q_pre, feed_dict={x: s1_t_r})
                    Q[0, a_r] =  np.max(Q_next)
                  print(" Q(st+1,at+1)를 구하는 W값")
                  print(sess.run([W1]))
                  print(" St+1_r{}".format(s1_t_r))
                  print(" 목표값{}".format(Q))
                  feed={x: s_t_r, y: Q}
                  sess.run(train, feed_dict=feed)
                  print(" REPLY 훈련W값")
                  print(sess.run([W1]))
                  print("Loss 값: {}  ".format(sess.run(loss,feed_dict=feed)))

        rList.append(rall)
        print("Episode {} finished after {} timesteps with r={}. Running score: {}".format(episode, count, rall, np.mean(rList)))


    for episode in range(10):
        # state 초기화
        s = env.reset()

        rall = 0
        d = False
        count = 0
        # 에피소드가 끝나기 전까지 반복
        while not d:
            count += 1
            # state 값의 전처리
            s_t = np.reshape(s, [1, INPUT])

            # 현재 상태의 Q값을 에측
            Q = sess.run(Q_pre, feed_dict={x: s_t})
            a = np.argmax(Q)

            # 결정된 action으로 Environment에 입력
            s, r, d, _ = env.step(a)

            # 총 reward 합
            rall += r


        rList.append(rall)

        print("Episode : {} steps : {} r={}. averge reward : {}".format(episode, count, rall,
                                                                        np.mean(rList)))


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Q(st,at): [[-0.00526684  0.0472389 ]]
Q(st,a)를 구하는 W값
[array([[ 0.80063725,  0.42707062],
       [ 0.16988516, -0.75265   ],
       [-0.27752447, -0.9358046 ],
       [-0.4266255 , -0.70764184]], dtype=float32)]
s_t: [[-0.02735222 -0.0264493  -0.00835379 -0.04408394]] action: 1 Reward1.0 
st+1: [-0.0278812   0.16879143 -0.00923547 -0.33939081]  doneFalse
메모리 [[array([[-0.02735222, -0.0264493 , -0.00835379, -0.04408394]]), 1, 1.0, array([-0.0278812 ,  0.16879143, -0.00923547, -0.33939081]), False]]
메모리 []
메모리 []
메모리 []
메모리 []
메모리 []
 EPISODE 0
Q(st,at): [[-0.00526684  0.0472389 ]]
Q(st,a)를 구하는 W값
[array([[ 0.80063725,  0.42707062